# Create z-score files

This notebook creates mean/std files for the paper. Resdidual coefficients are defined in a separated notebook.

In [1]:
import os
import yaml
import numpy as np
import xarray as xr

## File creation

In [2]:
# get variable information from data_preprocessing/config
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [3]:
N_levels = 13 # 13 levels

# get the 1979 training data for p_level info
base_dir = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_1deg/'
ds_example = xr.open_zarr(base_dir+'upper_subset/ERA5_subset_1deg_6h_1979_conserve.zarr')
level = np.array(ds_example['level'])

In [4]:
# get variable names
varnames = list(conf['zscore'].keys())
varnames = varnames[:-3] # remove save_loc and others

varname_surf = list(set(varnames) - set(['U', 'V', 'T', 'Q', 'Z', 'specific_total_water']))
varname_upper = ['U', 'V', 'T', 'Q', 'Z', 'specific_total_water']

# collect computed mean and variance values
# See "qsub_STEP01_compute_mean_std.ipynb"
MEAN_values = {}
STD_values = {}

for varname in varname_surf:
    save_name = conf['zscore']['save_loc'] + '{}_mean_std_{}.npy'.format(
        conf['zscore']['prefix'], varname)
    
    mean_std = np.load(save_name)
    MEAN_values[varname] = mean_std[0]
    STD_values[varname] = mean_std[1]

for varname in varname_upper:

    # -------------------------------------------- #
    # allocate all levels
    mean_std_all_levels = np.empty((2, N_levels))
    mean_std_all_levels[...] = np.nan
    
    for i_level in range(N_levels):
        save_name = conf['zscore']['save_loc'] + '{}_level{}_mean_std_{}.npy'.format(
            conf['zscore']['prefix'], i_level, varname)
        
        mean_std = np.load(save_name)
        mean_std_all_levels[:, i_level] = mean_std

    # -------------------------------------------- #
    # save
    MEAN_values[varname] = np.copy(mean_std_all_levels[0, :])
    STD_values[varname] = np.copy(mean_std_all_levels[1, :])

### Mean file

In [6]:
# ------------------------------------------------------- #
# Initialize dataset
ds_mean_6h = xr.Dataset(coords={"level": level})

for varname, data in MEAN_values.items():
    if len(data.shape) == 1:
        data_array = xr.DataArray(
            data,
            dims=["level",],
            coords={"level": level},
            name=varname,
        )
        ds_mean_6h[varname] = data_array
    else:
        data_array = xr.DataArray(
            data,
            name=varname,
        )
        ds_mean_6h[varname] = data_array

In [ ]:
ds_mean_6h['land_sea_CI_mask'] = 0.0 # land-sea mask uses 0.0

In [ ]:
# ds_mean_6h.to_netcdf(base_dir+'mean_std/mean_6h_1979_2019_subset_1deg.nc')

### Std file

In [7]:
# ------------------------------------------------------- #
# create xr.DataArray for std

# use the same level coord as mean
ds_std_6h = xr.Dataset(coords={"level": level})

for varname, data in STD_values.items():
    data = np.sqrt(data)
    if len(data.shape) == 1:
        data_array = xr.DataArray(
            data,
            dims=["level",],
            coords={"level": level},
            name=varname,
        )
        ds_std_6h[varname] = data_array
    else:
        data_array = xr.DataArray(
            data,
            name=varname,
        )
        ds_std_6h[varname] = data_array

In [9]:
# ds_std_6h.to_netcdf(base_dir+'mean_std/std_6h_1979_2019_subset_1deg.nc')

### Comparing with the old mean/std

In [10]:
# ------------------------------------------------------- #
# Compare with my old ones
STD_conserve = xr.open_dataset(base_dir+'mean_std/std_6h_1979_2019_subset_1deg.nc')
STD_bilinear = xr.open_dataset(base_dir+'mean_std/std_6h_1979_2019_bilinear_1deg.nc')

for varname in varnames:
    print('=============== {} ================='.format(varname))
    print(np.array(STD_conserve[varname]))
    print(np.array(STD_bilinear[varname]))

=============== MSL =================
1330.8846696194373
1332.1479128381566
=============== SKT =================
22.404838153788788
22.447001334275868
=============== SP =================
9597.442621657625
9656.6245797302
=============== TCC =================
0.34918409441572806
0.3658749665813759
=============== VAR_10U =================
5.487526636552918
5.52748270314236
=============== VAR_10V =================
4.71086288032973
4.744373030500121
=============== VAR_2T =================
21.373759417572977
21.40206776071968
=============== evaporation =================
0.0006418363022945283
0.0006502217778641388
=============== surface_latent_heat_flux =================
1604157.7576941664
1625160.551704857
=============== surface_net_solar_radiation =================
4214109.662542443
4226444.322152559
=============== surface_net_thermal_radiation =================
666310.6177924678
677867.5588998266
=============== surface_sensible_heat_flux =================
917596.9043234516
94243